In [1]:
%load_ext autoreload
%autoreload 2

import os, sys
import copy
import socket
from tqdm import tqdm
import torch
import pickle
from torch import optim
from pathlib import Path
from torch.utils.tensorboard import SummaryWriter

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../../")))
from libs import fl, nn, agg, data, poison, log

In [2]:
# Save Logs To File (info | debug | warning | error | critical) [optional]
log.init("info")
#log.init("info", "federated.log")
#log.init("debug", "flkafka.log")

In [3]:
class FedArgs():
    def __init__(self):
        self.num_clients = 50
        self.epochs = 10
        self.local_rounds = 1
        self.client_batch_size = 32
        self.test_batch_size = 128
        self.learning_rate = 1e-4
        self.weight_decay = 1e-5
        self.cuda = False
        self.seed = 1
        self.tb = SummaryWriter('../../out/runs/federated/FLTrust', comment="Mnist Centralized Federated training")

fedargs = FedArgs()

In [4]:
use_cuda = fedargs.cuda and torch.cuda.is_available()
torch.manual_seed(fedargs.seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

In [5]:
host = socket.gethostname()
clients = [host + "(" + str(client + 1) + ")" for client in range(fedargs.num_clients)]

In [6]:
#Initialize Global and Client models
global_model = nn.ModelMNIST()
client_models = {client: copy.deepcopy(global_model) for client in clients}

# Function for training
def train_model(_model, train_loader, fedargs, device):
    model, loss = fl.client_update(_model,
                                train_loader,
                                fedargs.learning_rate,
                                fedargs.weight_decay,
                                fedargs.local_rounds,
                                device)
    model_update = agg.sub_model(_model, model)
    return model_update, model, loss

In [7]:
# Load MNIST Data to clients
train_data, test_data = data.load_dataset("mnist")

In [8]:
# For securing if the next cell execution is skipped
FLTrust = None

<h1>FLTrust: Skip section below for any other averaging than FLTrust.</h1>

In [9]:
FLTrust = True
root_ratio = 0.01
train_data, root_data = torch.utils.data.random_split(train_data, [int(len(train_data) * (1-root_ratio)), 
                                                              int(len(train_data) * root_ratio)])
root_loader = torch.utils.data.DataLoader(root_data, batch_size=fedargs.client_batch_size, shuffle=True, **kwargs)

<h2>Resume</h2>

In [10]:
clients_data = data.split_data(train_data, clients)

<h1>Poison: Skip section below to run normal, modify if required.</h1>

In [11]:
for client in range(10):
    clients_data[clients[client]] = poison.label_flip(clients_data[clients[client]], 4, 9, poison_percent = -1)
    
#clients_data[clients[0]] = poison.label_flip(clients_data[clients[0]], 6, 2, poison_percent = 1)
#clients_data[clients[0]] = poison.label_flip(clients_data[clients[0]], 3, 8, poison_percent = 1)
#clients_data[clients[0]] = poison.label_flip(clients_data[clients[0]], 1, 5, poison_percent = 1)

<h2>Resume</h2>

In [12]:
client_train_loaders, _ = data.load_client_data(clients_data, fedargs.client_batch_size, None, **kwargs)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=fedargs.test_batch_size, shuffle=True, **kwargs)

clients_info = {
        client: {"train_loader": client_train_loaders[client]}
        for client in clients
    }

In [13]:
import asyncio
import time

def background(f):
    def wrapped(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, f, *args, **kwargs)

    return wrapped

@background
def process(client, epoch, model, train_loader, fedargs, device):
    # Train
    model_update, model, loss = train_model(model, train_loader, fedargs, device)

    #Plot and Log
    for local_epoch, loss in enumerate(list(loss.values())):
        fedargs.tb.add_scalars("Training Loss/" + client, {str(epoch): loss}, str(local_epoch + 1))

    log.jsondebug(loss, "Epoch {} of {} : Federated Training loss, Client {}".format(epoch, fedargs.epochs, client))
    log.modeldebug(model_update, "Epoch {} of {} : Client {} Update".format(epoch, fedargs.epochs, client))
    
    return model_update

In [14]:
import time
start_time = time.time()
    
# Federated Training
for _epoch in tqdm(range(fedargs.epochs)):

    epoch = _epoch + 1
    log.info("Federated Training Epoch {} of {}".format(epoch, fedargs.epochs))

    # Gloabal Model Update
    if epoch > 1:
        # For Tmean, not impacts others as of now
        avgargs = {"beta": 10}
        
        # For FLTrust, if FLTrust section is skipped, this piece of code will be ignored automatically
        if FLTrust:
            global_model_update, _, _ = train_model(global_model, root_loader, fedargs, device)
            avgargs["base_update"] = global_model_update
        
        # Average
        global_model = fl.federated_avg(client_model_updates, global_model, agg.Rule.FLTrust, **avgargs)
        log.modeldebug(global_model, "Epoch {} of {} : Server Update".format(epoch, fedargs.epochs))

        # Test
        global_test_output = fl.eval(global_model, test_loader, device, 4, 9)
        fedargs.tb.add_scalars("Test Evaluation", {
            'Gloabl Accuracy': global_test_output["accuracy"],
            'Attack Success Rate': global_test_output["attack"]["attack_success_rate"],
            'Misclassification Rate': global_test_output["attack"]["misclassification_rate"],
        }, epoch)
        log.jsoninfo(global_test_output, "Gloabl Test Outut after Epoch {} of {}".format(epoch, fedargs.epochs))
    
        # Update client models
        client_models = {client: copy.deepcopy(global_model) for client in clients}

    # Clients
    loop = asyncio.get_event_loop()
    tasks = [process(client, epoch, client_models[client],
                     clients_info[client]['train_loader'],
                     fedargs, device) for client in clients]
    updates = loop.run_until_complete(asyncio.gather(*tasks))
    client_model_updates = {client: update for client, update in zip(clients, updates)}
    
print(time.time() - start_time)

 10%|█         | 1/10 [00:31<04:44, 31.57s/it]2021-08-29 09:46:03,778 - <ipython-input-14-5f39cab2337c>::<module>(l:8) : Federated Training Epoch 2 of 10 [MainProcess : MainThread (INFO)]
/home/harsh_1921cs01/hub/F3IA/fl/libs/sim.py:48: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  _param_list = nd.array(param_list).squeeze()
2021-08-29 09:46:11,535 - /home/harsh_1921cs01/hub/F3IA/fl/libs/agg.py::FLTrust(l:90) : FLTrust Score [0.136, 0.15, 0.139, 0.131, 0.146, 0.134, 0.14, 0.14, 0.137, 0.144, 0.489, 0.525, 0.498, 0.525, 0.531, 0.518, 0.508, 0.507, 0.507, 0.531, 0.524, 0.546, 0.505, 0.53, 0.498, 0.509, 0.505, 0.52, 0.51, 0.512, 0.531, 0.487, 0.563, 0.523, 0.467, 0.478, 0.535, 0.539, 0.504, 0.509, 0.489, 0.504, 0.494, 0.499, 0.493, 0.498, 0.504, 0.508, 0.492, 0.475] [Mai

KeyboardInterrupt: 